# Pinecone DB (프리티어 사용)

In [ ]:
# !pip install pinecone

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

pinecone_api_key = os.getenv('PINECONE_API_KEY')

In [6]:
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(api_key=pinecone_api_key)

index_name = 'reviewtest'

pc.create_index(
    name=index_name,
    dimension=384,         # embedding 차원 수
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "reviewtest",
    "metric": "cosine",
    "host": "reviewtest-bhrdsje.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [3]:
reviews = [
    {'id': '1', 'text': '이 노트북 성능이 아주 뛰어나요! 빠르고 안정적이에요.', 'sentiment': 'positive'},
    {'id': '2', 'text': '디자인은 괜찮은데 성능이 기대보다 떨어져요.', 'sentiment': 'negative'},
    {'id': '3', 'text': '가격 대비 너무 좋은 성능이에요. 화면도 선명하고 좋습니다.', 'sentiment': 'positive'},
    {'id': '4', 'text': '배터리 수명이 짧아서 불편하네요. 좀 더 오래 가면 좋겠어요.', 'sentiment': 'negative'},
    {'id': '5', 'text': '디자인도 예쁘고 성능도 괜찮아요. 하루종일 사용해도 문제 없어요.', 'sentiment': 'positive'},
    {'id': '6', 'text': '노트북이 너무 무겁고 자꾸 열이 올라서 불편합니다.', 'sentiment': 'negative'},
    {'id': '7', 'text': '발열이 생각보다 적고, 쿨링 성능이 좋아요. 매우 만족합니다.', 'sentiment': 'positive'},
    {'id': '8', 'text': '전체적인 성능은 괜찮지만, 소음이 너무 심해서 일하는데 방해가 돼요.', 'sentiment': 'negative'},
    {'id': '9', 'text': '노트북이 가볍고 디자인도 세련되었어요. 화면이 정말 선명하고 좋아요.', 'sentiment': 'positive'},
    {'id': '10', 'text': '성능이 생각보다 느리고, 여러 프로그램을 동시에 켰을 때 멈춰요.', 'sentiment': 'negative'},
    {'id': '11', 'text': '가격에 비해 성능이 너무 좋아서 놀랐어요. 아주 만족합니다!', 'sentiment': 'positive'},
    {'id': '12', 'text': '키보드 타건감이 너무 딱딱하고 불편해요.', 'sentiment': 'negative'},
    {'id': '13', 'text': '터치패드가 너무 민감해서 실수로 클릭이 자주 발생해요.', 'sentiment': 'negative'},
    {'id': '14', 'text': '간단한 작업에서 아주 잘 작동하고, 디자인이 고급스러워요.', 'sentiment': 'positive'},
    {'id': '15', 'text': '노트북을 열 때마다 팬 소음이 너무 커서 불편해요.', 'sentiment': 'negative'},
    {'id': '16', 'text': '가격이 저렴하고 성능도 괜찮습니다. 가성비 좋은 노트북이에요.', 'sentiment': 'positive'},
    {'id': '17', 'text': '무게가 너무 가볍고, 화면 크기도 적당해서 휴대성도 뛰어나요.', 'sentiment': 'positive'},
    {'id': '18', 'text': '열이 많이 나는 편이라 오랫동안 사용하기 힘들어요.', 'sentiment': 'negative'},
    {'id': '19', 'text': '디자인은 괜찮지만 화면 색감이 기대보다 안 좋았어요.', 'sentiment': 'negative'},
    {'id': '20', 'text': '가성비 좋은 노트북이에요. 기본적인 작업에 아주 적합합니다.', 'sentiment': 'positive'}
]

In [7]:
# Pinecone index 및 임베딩 모델 로드
from sentence_transformers import SentenceTransformer

pc = Pinecone(api_key=pinecone_api_key)
idx = pc.Index(index_name)

model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# index에 리뷰 데이터 추가
for review in reviews:
    review_embed = model.encode(review['text']).tolist()
    idx.upsert([        # upsert: 해당 데이터가 있다면 update, 없다면 insert
        (
            review['id'],       # 문자열 형태의 id
            review_embed,
            {                   # dict 형태에 원문의 텍스트와 감정을 담게 해줌  # chromadb에서의 metadata와 같은 내용을 넣어주는거임
                'text': review['text'],     
                'sentiment': review['sentiment']
            }
        )
    ])

### 리뷰 데이터 조회

In [11]:
query_text = '이 노트북의 성능은 괜찮은가요?'
query_embed = model.encode(query_text).tolist()

# 질의 메서드
results = idx.query(
    vector=query_embed,
    top_k=3,
    include_metadata=True
)

results 
# matches: query_text와 유사도가 높은 애를 가지고 있음 -> 상위 3개

{'matches': [{'id': '2',
              'metadata': {'sentiment': 'negative',
                           'text': '디자인은 괜찮은데 성능이 기대보다 떨어져요.'},
              'score': 0.761447,
              'values': []},
             {'id': '16',
              'metadata': {'sentiment': 'positive',
                           'text': '가격이 저렴하고 성능도 괜찮습니다. 가성비 좋은 노트북이에요.'},
              'score': 0.730541587,
              'values': []},
             {'id': '5',
              'metadata': {'sentiment': 'positive',
                           'text': '디자인도 예쁘고 성능도 괜찮아요. 하루종일 사용해도 문제 없어요.'},
              'score': 0.705470443,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [15]:
for review in results['matches']:
    metadata = review['metadata']
    print(f"[감성: {metadata['sentiment']}]", metadata['text'])

[감성: negative] 디자인은 괜찮은데 성능이 기대보다 떨어져요.
[감성: positive] 가격이 저렴하고 성능도 괜찮습니다. 가성비 좋은 노트북이에요.
[감성: positive] 디자인도 예쁘고 성능도 괜찮아요. 하루종일 사용해도 문제 없어요.


### 리뷰 감성 분석

In [16]:
query_text = '배터리도 빨리 닳고 벽돌보다 무거워요. 비추천'
query_embed = model.encode(query_text).tolist()

# 질의 메서드
results = idx.query(
    vector=query_embed,
    top_k=5,
    include_metadata=True
)

results 

{'matches': [{'id': '10',
              'metadata': {'sentiment': 'negative',
                           'text': '성능이 생각보다 느리고, 여러 프로그램을 동시에 켰을 때 멈춰요.'},
              'score': 0.862432897,
              'values': []},
             {'id': '12',
              'metadata': {'sentiment': 'negative',
                           'text': '키보드 타건감이 너무 딱딱하고 불편해요.'},
              'score': 0.813677,
              'values': []},
             {'id': '4',
              'metadata': {'sentiment': 'negative',
                           'text': '배터리 수명이 짧아서 불편하네요. 좀 더 오래 가면 좋겠어요.'},
              'score': 0.807199538,
              'values': []},
             {'id': '16',
              'metadata': {'sentiment': 'positive',
                           'text': '가격이 저렴하고 성능도 괜찮습니다. 가성비 좋은 노트북이에요.'},
              'score': 0.779535,
              'values': []},
             {'id': '13',
              'metadata': {'sentiment': 'negative',
                           'text': '터치패드가 너무 민감해서 실수로 클릭이 자주 발생해요.'},
 

In [18]:
sentiment_counts = {
    'positive':0,
    'negative':0
}

for review in results['matches']:
    sentiment = review['metadata']['sentiment']
    sentiment_counts[sentiment] += 1

print(f"리뷰 분석 결과: {'positive' if sentiment_counts['positive'] > sentiment_counts['negative'] else 'negative'}")

리뷰 분석 결과: negative
